# Running time code

### **This notebook shows how long the source code takes to get the final output. Here, we use all functions in *Gravmag* and also *Data filtering*.**

In [1]:
# Copying all usefull libraries
import numpy as np
import gravmag as gmc
import datafiltering as df
import time

### Define all discription for running the codes, like number of points and models.

In [2]:
# Definition for number of points and grid creation
n = 500
x = np.linspace(-1000., 1000., n)
y = x
z = np.random.normal(loc = -50., scale = 1., size = n)
# Creat the grid file
X, Y = np.meshgrid(x, y)

### Define both synthetic models: prism and sphere.

In [3]:
spheregrav = (0., 0., 500., 500., 2673.) 
spheremag = (0., 0., 500., 500., 2.25) 
prismgrav = (-250., 250., -250., 250., 100., 300., 2673.)
prismmag = (-250., 250., -250., 250., 100., 300., 2.250)

**(1) Gravity sources**

In [8]:
print 'By using a rectangular prism:'
print 'Gravitational potential'
%timeit -n 10 gmc.potential(X, Y, z, prismgrav)
print 'Gx component:'
%timeit -n 10 gmc.prism_gx(X, Y, z, prismgrav)
print 'Gy component:'
%timeit -n 10 gmc.prism_gy(X, Y, z, prismgrav)
print 'Gz component:'
%timeit -n 10 gmc.prism_gz(X, Y, z, prismgrav)
print
print 'By using a solid sphere:'
print 'Gz component:'
%timeit -n 10 gmc.potential(X, Y, z, prismgrav)

By using a rectangular prism:
Gravitational potential
10 loops, best of 3: 923 ms per loop
Gx component:
10 loops, best of 3: 311 ms per loop
Gy component:
10 loops, best of 3: 325 ms per loop
Gz component:
10 loops, best of 3: 365 ms per loop

By using a solid sphere:
Gz component:
10 loops, best of 3: 907 ms per loop


**(2) Magnetic sources**

In [10]:
print 'By using a rectangular prism:'
print 'Total field anomaly'
%timeit -n 10 gmc.prism_tf(X, Y, z, prismmag, (42., 22.), (42., 22.))
print
print 'By using a solid sphere:'
print 'Bx component:'
%timeit -n 10 gmc.sphere_bx(X, Y, z, spheremag, (42., 22.))
print 'By component:'
%timeit -n 10 gmc.sphere_by(X, Y, z, spheremag, (42., 22.))
print 'Bz component:'
%timeit -n 10 gmc.sphere_bz(X, Y, z, spheremag, (42., 22.))
print 'True total field anomaly:'
%timeit -n 10 gmc.sphere_tf(X, Y, z, spheremag, (42., 22.), (42., 22.), 23500.)
print 'Approximated total field anomaly:'
%timeit -n 10 gmc.sphere_tf(X, Y, z, spheremag, (42., 22.), (42., 22.), 23500.)

By using a rectangular prism:
Total field anomaly
10 loops, best of 3: 845 ms per loop

By using a solid sphere:
Bx component:
10 loops, best of 3: 56.1 ms per loop
By component:
10 loops, best of 3: 56.5 ms per loop
Bz component:
10 loops, best of 3: 56.1 ms per loop
True total field anomaly:
10 loops, best of 3: 192 ms per loop
Approximated total field anomaly:
10 loops, best of 3: 179 ms per loop


**Transform and filters for potential data**